In [2]:
import pandas as pd
import numpy as np
import os 
import re
pd.set_option('display.max_columns', None)
from scipy.optimize import bisect
from sklearn.metrics import log_loss

In [3]:
df = pd.read_csv('../../../data/datasets_for_analysis/final_datasets/odds_w_outcomes_one_row_per_fight.csv')
df.head()

,fighter1,fighter2,fight_odds_url,Bet,5Dimes_win,Bet365_win,BetDSI_win,BetOnline_win,BookMaker_win,Bovada_win,Intertops_win,Pinnacle_win,SportBet_win,SportsInt_win,Sportsbook_win,William_H_win,meanodds_win,5Dimes_vig,BetDSI_vig,BookMaker_vig,SportBet_vig,Bet365_vig,Bovada_vig,Sportsbook_vig,William_H_vig,Pinnacle_vig,SportsInt_vig,BetOnline_vig,Intertops_vig,Winner_Cleaned,Loser_Cleaned,wiki_url,bestfightodds_winner,bestfightodds_loser,WeightClass,Winner,Outcome,Loser,Method,Round,Time,Notes,Card,Winner_url,Loser_url,event_order,Method_Cleaned,Card_Cleaned,Champion,Interim_Champion,Card_Date,#,Event,5Dimes_lose,BetDSI_lose,BookMaker_lose,SportBet_lose,Bet365_lose,Bovada_lose,Sportsbook_lose,William_H_lose,Pinnacle_lose,SportsInt_lose,BetOnline_lose,Intertops_lose,meanodds_lose
0,Aalon Cruz,Spike Carlyle,https://www.bestfightodds.com/events/ufc-on-es...,Spike Carlyle,2.700000,2.5,2.630000,2.600000,2.580000,2.600000,2.5,2.650000,2.690000,2.450000,2.500000,2.6,2.583333,0.019493,0.042390,0.054264,0.022097,0.036364,0.051282,0.049123,0.051282,0.037222,0.051020,0.033738,0.055172,Spike Carlyle,Aalon Cruz,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,Spike Carlyle,Aalon Cruz,Featherweight,Spike Carlyle,def,Aalon Cruz,TKO (punches),1.0,1:25,NaN,Preliminary card (ESPN+),NaN,NaN,11,KO,Prelim,NaN,NaN,2020-02-29,511,UFC Fight Night: Benavidez vs. Figueiredo,1.540541,1.510204,1.500000,1.537634,1.571429,1.500000,1.540541,1.5,1.515464,1.555556,1.540541,1.526316,1.528185
1,Aaron Phillips,Matt Hobar,https://www.bestfightodds.com/events/ufc-fight...,Matt Hobar,1.487805,NaN,1.476190,1.476190,1.476190,1.444444,NaN,1.487805,1.476190,1.450450,1.444444,NaN,1.468857,0.019353,0.034562,0.034562,0.030776,NaN,0.055944,0.049451,NaN,0.024244,0.059811,0.034562,NaN,Matt Hobar,Aaron Phillips,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,Matt Hobar,Aaron Phillips,Bantamweight,Matt Hobar,def,Aaron Phillips,"Decision (unanimous) (29–28, 29–28, 29–28)",3.0,5:00,NaN,Preliminary card (Fox Sports 2),https://en.wikipedia.org/wiki/Matt_Hobar,NaN,9,Unanimous Decision,Prelim,NaN,NaN,2014-08-23,287,UFC Fight Night: Henderson vs. dos Anjos,2.880000,2.800000,2.800000,2.830000,NaN,2.750000,2.800000,NaN,2.840000,2.700000,2.800000,NaN,2.800000
2,Aaron Phillips,Sam Sicilia,https://www.bestfightodds.com/events/ufc-173-b...,Sam Sicilia,1.714286,NaN,1.689655,1.666667,1.689655,1.645161,NaN,1.684932,1.704225,1.609756,1.625000,NaN,1.669926,0.018116,0.036281,0.036281,0.025373,NaN,0.052288,0.050167,NaN,0.024530,0.055995,0.034783,NaN,Sam Sicilia,Aaron Phillips,https://en.wikipedia.org/wiki/UFC_173,Sam Sicilia,Aaron Phillips,Featherweight,Sam Sicilia,def,Aaron Phillips,"Decision (unanimous) (29–28, 29–28, 30–27)",3.0,5:00,NaN,Preliminary card (UFC Fight Pass),https://en.wikipedia.org/wiki/Sam_Sicilia,NaN,11,Unanimous Decision,Prelim,NaN,NaN,2014-05-24,273,UFC 173: Barão vs. Dillashaw,2.300000,2.250000,2.250000,2.280000,NaN,2.250000,2.300000,NaN,2.320000,2.300000,2.300000,NaN,2.283333
3,Aaron Riley,Joe Brammer,https://www.bestfightodds.com/events/ufc-114-j...,Aaron Riley,1.526316,NaN,NaN,NaN,1.512821,1.540541,NaN,1.507614,1.526316,NaN,1.540541,NaN,1.525691,0.032531,NaN,0.053174,0.032531,NaN,0.041280,0.041280,NaN,0.024310,NaN,NaN,NaN,Aaron Riley,Joe Brammer,https://en.wikipedia.org/wiki/UFC_114,Aaron Riley,Joe Brammer,Lightweight,Aaron Riley,def,Joe Brammer,"Decision (unanimous) (30–27, 30–27, 30–27)",3.0,5:00,NaN,Preliminary card,https://en.wikipedia.org/wiki/Aaron_Riley,NaN,10,Unanimous Decision,Prelim,NaN,NaN,2010-05-29,152,UFC 114: Rampage vs. Evans,2.650000,NaN,2.550000,2.650000,NaN,2.550000,2.550000,NaN,2.770000,NaN,NaN,NaN,2.620000
4,Aaron Riley,Jorge Gurgel,https://www.bestfightodds.com/events/ufc-91-co...,Aaron Riley,1.952381,NaN,NaN,NaN,1.909091,1.869565,NaN,2.100000,1.952381,2.000000,1.952381,NaN,1.962257,0.047079,NaN,0.069264,0.067751,NaN,0.069767,0.067751,NaN,0.021645,0.081590,NaN,NaN,Aaron Riley,Jorge Gurgel,https://en.wikipedia.org/wiki/UFC_91,Aaron Riley,Jorge Gurgel

In [4]:
# create functions for converting a pair of decimal odds 
# for 2 mutually exclusive outcomes
# to true probabilities by removing the overround

def basic(x):
    out = x**-1
    return out/np.sum(out)

def shin(x):
    out = x**-1
    vig_half = (np.sum(out) -1.0)/2.0
    return out - vig_half

def wpo(x):
    vig = np.sum(x**- 1.0) - 1.0
    return (2.0 - vig * x)/(2.0*x)


def oddsratio(x):
    breakeven = x**-1
    odds =  breakeven/(1-breakeven)
    odds1 = np.sqrt(odds[0]/odds[1])
    prob = odds1/(1+odds1)
    return np.array([prob, 1.0 - prob])

def powersolver(k, probs):
    return np.sum(probs**(1/k)) -1.0
    
def powermethod(x):
    breakeven = x**-1
    k = bisect(powersolver, .001, 1, args=(breakeven, ))
    return breakeven**(1/k)

def bb(x):
    inverted_odds = x**-1
    zz = np.sum(inverted_odds) - 1.0
    return (inverted_odds - zz)/(1-zz)

def implied(x, method):
    
    if x.isna().any():
        # assumption is if one element is nan then both are
        return x
    
    if method == 'bb':
        s = bb(x)
    elif method == 'shin':
        s = shin(x)
    elif method == 'or':
        s = oddsratio(x)
    elif method == 'wpo':
        s = wpo(x)
    elif method == 'basic':
        s = basic(x)
    elif method == 'power':
        s = powermethod(x)
        
    return pd.Series(s, x.index)


In [5]:
# determine which method of removing overround is most effective
methods = ['or', 'power', 'basic', 'wpo', 'bb', 'shin']

cols = ['5Dimes','BetDSI','BookMaker','SportBet','Bet365','Bovada', 
                'Sportsbook','William_H','Pinnacle',
       'SportsInt','BetOnline','Intertops', 'meanodds']

for method in methods:
    for col in cols:
        vals = df[[col+'_win', col+'_lose']].apply(lambda x: implied(x, method = method), axis = 1)
        df[col+'_win'+'_'+method] = vals[col+'_win']

In [6]:
df = df[df.Outcome == 'def']

In [7]:
logloss_df = pd.DataFrame(columns = ['method', 'betsite', 'logloss'])
logloss_df['length'] = range(len(methods)*len(cols))
del logloss_df['length']

index = 0
for method in methods:
    for col in cols:
        logloss_df['method'].iloc[index] = method
        logloss_df['betsite'].iloc[index] = col
        p = df[col+'_win'+'_'+method][df[col+'_win'+'_'+method].notna()].values.reshape(-1, 1)
        logloss_df['logloss'].iloc[index] = -np.mean(np.log(p))
        index += 1

In [8]:
pivoted = logloss_df.pivot_table(values = ['logloss'], columns = ['method'], 
                    index = 'betsite',
                    aggfunc='first').reset_index()
pivoted.columns = [a if b == '' else b for (a, b) in pivoted.columns] 
display(pivoted.rank(axis=1))
display(pivoted)

,basic,bb,or,power,shin,wpo
0,6.0,1.0,3.0,2.0,4.5,4.5
1,6.0,5.0,1.0,4.0,2.5,2.5
2,6.0,1.0,3.0,2.0,4.5,4.5
3,6.0,1.0,3.0,2.0,4.5,4.5
4,6.0,1.0,3.0,2.0,4.5,4.5
5,6.0,1.0,3.0,2.0,4.5,4.5
6,6.0,1.0,3.0,2.0,4.5,4.5
7,6.0,1.0,3.0,2.0,4.0,5.0
8,6.0,1.0,3.0,2.0,4.0,5.0
9,6.0,2.0,3.0,1.0,4.5,4.5


,betsite,basic,bb,or,power,shin,wpo
0,5Dimes,0.613688,0.613089,0.613314,0.613187,0.613331,0.613331
1,Bet365,0.628809,0.628716,0.628564,0.628572,0.628572,0.628572
2,BetDSI,0.614886,0.613756,0.614135,0.613906,0.614186,0.614186
3,BetOnline,0.614781,0.613830,0.614168,0.613968,0.614202,0.614202
4,BookMaker,0.614904,0.614024,0.614273,0.614104,0.614309,0.614309
5,Bovada,0.614695,0.613447,0.613779,0.613544,0.613839,0.613839
6,Intertops,0.620248,0.618283,0.618970,0.618564,0.619076,0.619076
7,Pinnacle,0.614303,0.613671,0.613904,0.613770,0.613921,0.613921
8,SportBet,0.614990,0.614547,0.614671,0.614584,0.614681,0.614681
9,SportsInt,0.615293,0.614081,0.614215,0.614019,0.614318,0.614318


In [10]:
# repeat analysis using brier score

brier_df = pd.DataFrame(columns = ['method', 'betsite', 'brier_score'])
brier_df['length'] = range(len(methods)*len(cols))
del brier_df['length']


index = 0
for method in methods:
    for col in cols:
        brier_df['method'].iloc[index] = method
        brier_df['betsite'].iloc[index] = col
        p = df[col+'_win'+'_'+method][df[col+'_win'+'_'+method].notna()].values.reshape(-1, 1)
        brier_df['brier_score'].iloc[index] = np.mean((1-p)**2)
        index += 1
pivoted = brier_df.pivot_table(values = ['brier_score'], columns = ['method'], 
                    index = 'betsite',
                    aggfunc='first').reset_index()
pivoted.columns = [a if b == '' else b for (a, b) in pivoted.columns] 
display(pivoted.rank(axis=1))
display(pivoted)

,basic,bb,or,power,shin,wpo
0,6.0,1.0,3.0,2.0,4.5,4.5
1,4.0,6.0,3.0,5.0,1.5,1.5
2,6.0,1.0,3.0,2.0,4.5,4.5
3,6.0,1.0,3.0,2.0,4.5,4.5
4,6.0,1.0,3.0,2.0,4.5,4.5
5,6.0,1.0,3.0,2.0,4.0,5.0
6,6.0,1.0,3.0,2.0,4.5,4.5
7,6.0,1.0,3.0,2.0,4.5,4.5
8,6.0,1.0,3.0,2.0,4.5,4.5
9,6.0,5.0,2.0,1.0,3.5,3.5


,betsite,basic,bb,or,power,shin,wpo
0,5Dimes,0.212854,0.212681,0.212741,0.212706,0.212746,0.212746
1,Bet365,0.220062,0.220183,0.220055,0.220092,0.220053,0.220053
2,BetDSI,0.213429,0.213108,0.213201,0.213140,0.213216,0.213216
3,BetOnline,0.213454,0.213206,0.213280,0.213231,0.213290,0.213290
4,BookMaker,0.213325,0.213091,0.213138,0.213097,0.213148,0.213148
5,Bovada,0.213209,0.212873,0.212935,0.212877,0.212952,0.212952
6,Intertops,0.215945,0.215393,0.215556,0.215449,0.215585,0.215585
7,Pinnacle,0.213154,0.212988,0.213042,0.213007,0.213047,0.213047
8,SportBet,0.213444,0.213337,0.213356,0.213337,0.213358,0.213358
9,SportsInt,0.213551,0.213309,0.213256,0.213235,0.213280,0.213280


## Looks like Balanced Book method is best
but the numbers are all quite close